# 문서 단어 행렬(Document-Term Matrix, DTM)
- 서로 다른 문서의 BoW를 결합

## 1. 문서 단어 행렬의 표기법
- 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현

- 예시
    - 문서1 : 먹고 싶은 사과
    - 문서2 : 먹고 싶은 바나나
    - 문서3 : 길고 노란 바나나 바나나
    - 문서4 : 저는 과일이 좋아요

![표1](./image1.png)

## 2. 문서 단어 행렬(Document-Term Matrix)의 한계
- 희소 표현
- 단순 빈도 수 기반 접근
    - the, a 가 많다고 해서 서로 다른 문서들을 비슷한 유형이라고 판단할 수 없다.
    - DTM에 불용어와 중요한 단어에 대하여 가중치를 줄 수 있는 방법? -> TF-IDF

## 3. TF-IDF (Term Frequency-Inverse Document Frequency)
- 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취하는 것)으로 각 단어의 중요 정도를 가중치로 준다.
- DTM을 만든 후 , TF-IDF 가중치 부여

1. tf(d,t): 특정 문서 d에서의 특정 단어 t의 등장 횟수
    - 각 단어들이 행렬에서 가지는 값 ( 등장 빈도 수)
2. df(t) : 특정 단어 t가 등장한 문서의 수
    - 특정 단어가, 특정 문서에 등장하는 횟수 
    - 단어의 빈도수가 아님
3. Idf(d,t) : df(t)에 반비례하는 수
$$ idf(d,t) = log(\frac{n}{1+df(t)})$$
    

In [1]:
import pandas as pd
from math import log

docs = [
    '먹고 싶은 사과',
    '먹고 싶은 바나나',
    '길고 노란 바나나 바나나',
    '저는 과일이 좋아요'
]
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [6]:
# 총 문서의 수
N = len(docs)

def tf(t,d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t,d):
    return tf(t,d) * idf(t)

In [8]:
result = []

for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t,d))
        
tf_ = pd.DataFrame(result, columns=vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [9]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [10]:
result = []
for i in range(N):
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


## 3. 사이킷런의 TF-IDF 활용하기

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
